In [ ]:
def column_fill_with_dummies(
    column: pd.Series,
    proportion_below: float=0.10,
    jitter: float=0.075,
    seed: int=42
) -> pd.Series:

    column = column.copy(deep=True)

    # Extract values metadata.
    missing_mask = column.isna()
    number_missing_values = missing_mask.sum()
    column_range = column.max() - column.min()

    # Shift data
    column_shift = column.min() - column.min() * proportion_below

    # Create the "jitter" (noise) to be added around the points.
    np.random.seed(seed)
    column_jitter = (np.random.rand(number_missing_values) - 2) * column_range * jitter

    # Save new dummy data.
    column[missing_mask] = column_shift + column_jitter

    return column


def xdxd():
    (
        riskfactors_df
        .select_columns("weight_lbs", "height_inch", "bmi")
        .missing.bind_shadow_matrix(true_string=True, false_string=False)
        .apply(
            axis="rows",
            func=lambda column: column.fillna(column.mean())
            if "_NA" not in column.name
            else column,
        )
        .pivot_longer(
            index="*_NA"
        )
        .pivot_longer(
            index=["variable", 'value'],
            names_to="variable_NA",
            values_to="value_NA"
        )
        .assign(
            valid=lambda df: df.apply(axis="columns", func=lambda column: column.variable in column.variable_NA)
        )
        .query("valid")
        .pipe(
            lambda df: (
                sns.displot(
                    data=df,
                    x="value",
                    hue="value_NA",
                    col="variable",
                    common_bins=False,
                    facet_kws={
                        "sharex": False,
                        "sharey": False
                    }
                )
            )
        )
    )